In [ ]:
%pip install duckdb

In [7]:
import duckdb

In [8]:
cn = duckdb.connect()
cn.sql("create or replace view v_detail_workouts as select * from read_csv_auto('~/concept2/workouts/*detail*.csv')")
cn.sql("create or replace view v_summary_workouts as select * from read_csv_auto('~/concept2/workouts/*summary*.csv')")

In [9]:
cn.sql("from v_detail_workouts limit 3").show()

sql = """
    select machine_type, count(*) as stroke_cnt
    , round(avg("stroke rate"),2) as avg_stroke_rate
    from v_detail_workouts
    group by 1
"""
cn.sql(sql).show()


┌────────┬────────────────┬───────────────────┬────────────────┬───────┬────────┬─────────────┬────────────┬────────┬────────────┬──────────────┐
│ Number │ Time (seconds) │ Distance (meters) │ Pace (seconds) │ Watts │ Cal/Hr │ Stroke Rate │ Heart Rate │ season │    date    │ machine_type │
│ int64  │     double     │      double       │     double     │ int64 │ int64  │    int64    │   int64    │ int64  │    date    │   varchar    │
├────────┼────────────────┼───────────────────┼────────────────┼───────┼────────┼─────────────┼────────────┼────────┼────────────┼──────────────┤
│      1 │            0.7 │               2.1 │            0.0 │  NULL │   NULL │        NULL │       NULL │   2024 │ 2024-04-03 │ rowerg       │
│      2 │            2.7 │               8.8 │          169.5 │    72 │    547 │        NULL │       NULL │   2024 │ 2024-04-03 │ rowerg       │
│      3 │            5.2 │              17.1 │          150.1 │   103 │    656 │          27 │       NULL │   2024 │ 2024-0

In [16]:
sql = """
    select 
        case 
            when grouping(season) = 1 and grouping(type) = 1 then 'Grand Total'
            when grouping(season) = 0 and grouping(type) = 1 then 'Sub Total'
            else cast(season as varchar)
        end as season_year,
        case 
            when grouping(type) = 1 then 'All Machines'
            else type
        end as machine_type,
        sum("Work Distance") as total_meters,
        round(avg("Drag Factor"),2) as avg_drag_factor,
        round(avg("Stroke Rate/Cadence"),2) as avg_stroke_rate
        ,sum("Stroke Count") as total_strokes
        ,sum("Work Time (Seconds)") as total_seconds
        ,sum("Total Cal") as total_calories
        ,sum("Total Cal")/3500 as fat_lbs_burned
    from v_summary_workouts
    group by rollup(season, type)
"""
cn.sql(sql).show()

cn.sql("from v_summary_workouts limit 5").show()

┌─────────────┬──────────────┬──────────────┬─────────────────┬─────────────────┬───────────────┬────────────────────┬────────────────┬────────────────────┐
│ season_year │ machine_type │ total_meters │ avg_drag_factor │ avg_stroke_rate │ total_strokes │   total_seconds    │ total_calories │   fat_lbs_burned   │
│   varchar   │   varchar    │    int128    │     double      │     double      │    int128     │       double       │     int128     │       double       │
├─────────────┼──────────────┼──────────────┼─────────────────┼─────────────────┼───────────────┼────────────────────┼────────────────┼────────────────────┤
│ 2026        │ SkiErg       │        31100 │            76.9 │            41.6 │          5517 │             8042.8 │           1871 │ 0.5345714285714286 │
│ Grand Total │ All Machines │      1150452 │          121.46 │           28.33 │         97306 │ 274550.30000000005 │          76601 │             21.886 │
│ 2025        │ RowErg       │       864733 │          129